In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [5]:
withdraw_transactions_df = pd.read_csv("../data/lighter_complete_withdraw_txs.csv")
deposit_transactions_df = pd.read_csv("../data/lighter_complete_deposit_txs.csv")

In [9]:
tornado_addresses = {
    "0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3": "100 DAI",
    "0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144": "1000 DAI",
    "0x07687e702b410fa43f4cb4af7fa097918ffd2730": "10000 DAI",
    "0x23773e65ed146a459791799d01336db287f25334": "100000 DAI",
    "0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc": "0.1 ETH",
    "0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936": "1 ETH",
    "0x910cbd523d972eb0a6f4cae4618ad62622b39dbf": "10 ETH",
    "0xa160cdab225685da1d56aa342ad8841c3b53f291": "100 ETH",
    "0xd96f2b1c14db8458374d9aca76e26c3d18364307": "100 USDC",
    "0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d": "1000 USDC",
    "0x169ad27a470d064dede56a2d3ff727986b15d52b": "100 USDT",
    "0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f": "1000 USDT",
    "0x178169b423a011fff22b9e3f3abea13414ddd0f1": "0.1 WBTC",
    "0x610b717796ad172b316836ac95a2ffad065ceab4": "1 WBTC",
    "0xbb93e510bbcd0b7beb5a853875f9ec60275cf498": "10 WBTC",
    "0x22aaa7720ddd5388a3c0a3333430953c68f1849b": "5000 cDAI",
    "0x03893a7c7463ae47d46bc7f091665f1893656003": "50000 cDAI",
    "0x2717c5e28cf931547b621a5dddb772ab6a35b701": "500000 cDAI",
    "0xd21be7248e0197ee08e0c20d4a96debdac3d20af": "5000000 cDAI"
}

In [37]:
pool_to_index_map = {
     "0.1 ETH": 0,
     "1 ETH": 1,
     "10 ETH": 2,
     "100 ETH": 3,
     "100 USDC": 4,
     "1000 USDC": 5,
     "100 USDT": 6,
     "1000 USDT": 7,
     "0.1 WBTC": 8,
     "1 WBTC": 9,
     "10 WBTC": 10,
     "5000 cDAI": 11,
     "50000 cDAI": 12,
     "500000 cDAI": 13,
     "5000000 cDAI": 14,
     "100 DAI": 15,
     "1000 DAI": 16,
     "10000 DAI": 17,
     "100000 DAI": 18
    }

In [26]:
tt = [0, 0]
tuple(tt)

(0, 0)

In [27]:
tt[a["0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936"]] += 1
tt

[1, 0]

In [14]:
withdraw_transactions_count_by_address = withdraw_transactions_df["from_address"].value_counts()
repeated_withdraw_transactions_by_address_count = withdraw_transactions_count_by_address[withdraw_transactions_count_by_address > 1].keys()
repeated_withdraw_transactions_by_address = withdraw_transactions_df[withdraw_transactions_df["from_address"].isin(repeated_withdraw_transactions_by_address_count)]
repeated_withdraw_transactions_by_address


,Unnamed: 0,Unnamed: 0.1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,...,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
0,0,7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb...,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,...,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be9...,NaN,NaN,NaN,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xd843ce0f9da3bd30537b9850711ab8089e06b4cf
1,1,8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da...,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,...,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac06...,NaN,NaN,NaN,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0xb77562124be8ac967cf7fc24573fe252aa39d95d
2,2,9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f...,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,...,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290...,NaN,NaN,NaN,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
3,3,13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad1...,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,...,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaee...,NaN,NaN,NaN,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x42a9cf901e889ca8a685394854c27898dbff86b0
4,4,15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed...,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,...,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e...,NaN,NaN,NaN,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xc9fb4b16800e61076326145e1066e876cf9f7a7b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83777,83777,181253,0x5a36df89fa1f1ef6de7254e4b5fbfd2d6c2297ef4621...,164,188,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570447,70188284576,...,1,2021-09-05 01:23:28 UTC,13162729,0xe14fb1085bc975af1ad751aa579d7de298fb5be86c3d...,1.027888e+11,2.210156e+09,2.0,70188284576,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47
83778,83778,181255,0x43a081968e109f71a2f807f4a94ffc77429b322c3c1e...,169,168,0xe7a784acb145bd8993e83d41adf43b7c7195e995,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567923,39255417592,...,1,2021-10-03 14:49:29 UTC,13346858,0xe5a4d9c829998f00873f6ac4603174d707ffb2ea79a3...,4.000000e+10,1.130179e+09,2.0,39255417592,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xe7a784acb145bd8993e83d41adf43b7c7195e995
83779,83779,181262,0xd0708b3e2135d0378a9d6fe3fdc58b3c4fbcbcbe926c...,246,10,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567971,50193886615,...,1,2021-10-17 23:36:24 UTC,13438417,0x36808310975d158e0f2b3a2f892abed25a955684f407...,7.518635e+10,1.519911e+09,2.0,50193886615,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4
83780,83780,181263,0x54b5ceca9fd3d2ff7893c32f901f3d1d5e2e4383607b...,127,180,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570459,81048817558,...,1,2021-09-04 22:53:33 UTC,13162025,0x15a807c50ecea90cd58f5f9d6a26e0c60f3e9f06dfa6...,1.349098e+11,3.899014e+09,2.0,81048817558,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47


In [15]:
withdraw_transactions_df

,Unnamed: 0,Unnamed: 0.1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,...,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
0,0,7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb...,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,...,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be9...,NaN,NaN,NaN,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xd843ce0f9da3bd30537b9850711ab8089e06b4cf
1,1,8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da...,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,...,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac06...,NaN,NaN,NaN,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0xb77562124be8ac967cf7fc24573fe252aa39d95d
2,2,9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f...,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,...,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290...,NaN,NaN,NaN,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
3,3,13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad1...,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,...,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaee...,NaN,NaN,NaN,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x42a9cf901e889ca8a685394854c27898dbff86b0
4,4,15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed...,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,...,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e...,NaN,NaN,NaN,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xc9fb4b16800e61076326145e1066e876cf9f7a7b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83777,83777,181253,0x5a36df89fa1f1ef6de7254e4b5fbfd2d6c2297ef4621...,164,188,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570447,70188284576,...,1,2021-09-05 01:23:28 UTC,13162729,0xe14fb1085bc975af1ad751aa579d7de298fb5be86c3d...,1.027888e+11,2.210156e+09,2.0,70188284576,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47
83778,83778,181255,0x43a081968e109f71a2f807f4a94ffc77429b322c3c1e...,169,168,0xe7a784acb145bd8993e83d41adf43b7c7195e995,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567923,39255417592,...,1,2021-10-03 14:49:29 UTC,13346858,0xe5a4d9c829998f00873f6ac4603174d707ffb2ea79a3...,4.000000e+10,1.130179e+09,2.0,39255417592,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xe7a784acb145bd8993e83d41adf43b7c7195e995
83779,83779,181262,0xd0708b3e2135d0378a9d6fe3fdc58b3c4fbcbcbe926c...,246,10,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567971,50193886615,...,1,2021-10-17 23:36:24 UTC,13438417,0x36808310975d158e0f2b3a2f892abed25a955684f407...,7.518635e+10,1.519911e+09,2.0,50193886615,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4
83780,83780,181263,0x54b5ceca9fd3d2ff7893c32f901f3d1d5e2e4383607b...,127,180,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570459,81048817558,...,1,2021-09-04 22:53:33 UTC,13162025,0x15a807c50ecea90cd58f5f9d6a26e0c60f3e9f06dfa6...,1.349098e+11,3.899014e+09,2.0,81048817558,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47


### Function summary: compare_transactions

Given two dictionaries, withdraw_dict and deposit_dict representing the total deposits and withdraws made by an address to each TCash pool, respectively, compares if the set of keys of both are equal and when they are, checks if all values in the deposit dictionary are equal or greater than each of the corresponding values of the withdraw dicionary. If this is the case, returns $True$, if not, $False$.

In [4]:
def compare_transactions(withdraw_dict, deposit_dict):
    if set(withdraw_dict.keys()) != set(deposit_dict.keys()):
        return False
    for currency in withdraw_dict.keys():
        if not (deposit_dict[currency] >= withdraw_dict[currency]):
            return False
    return True

### Function summary: get_number_of_withdraws

Given a particular withdraw transaction and the withdraw transactions DataFrame, gets the total withdraws the address made in each pool. It is returned as a dictionary with the pools as the keys and the number of withdraws as the values.

In [5]:
def get_number_of_withdraws(withdraw_transaction, withdraw_transactions_df, tornado_addresses):
    
    n_withdraws = {tornado_addresses[withdraw_transaction.to_address]: 1}
    
    for withdraw_row in withdraw_transactions_df.itertuples():
        if (withdraw_row.recipient_address == withdraw_transaction.recipient_address) and (withdraw_row.block_timestamp <= withdraw_transaction.block_timestamp) and (withdraw_row.hash != withdraw_transaction.hash):
            if tornado_addresses[withdraw_row.tornado_cash_address] in n_withdraws.keys():
                n_withdraws[tornado_addresses[withdraw_row.tornado_cash_address]] += 1
            else:
                n_withdraws[tornado_addresses[withdraw_row.tornado_cash_address]] = 1
    return n_withdraws

### Function summary: get_address_deposits

Given the deposit transactions DataFrame, returns a dictionary with every address of the deposit

In [7]:
def get_address_deposits(deposit_transactions_df, tornado_addresses):
    
    addresses_and_deposits_counts = pd.DataFrame(deposit_transactions_df[["from_address", "tornado_cash_address"]].value_counts()).rename(columns={0:"count"})
    
    addresses_and_deposits_dict = {}
    for row in addresses_and_deposits_counts.itertuples():
        if row.Index[0] in addresses_and_deposits_dict.keys():
            addresses_and_deposits_dict[row.Index[0]][tornado_addresses[row.Index[1]]] = row.count
        else:
            addresses_and_deposits_dict[row.Index[0]] = {tornado_addresses[row.Index[1]]: row.count}
            
    return addresses_and_deposits_dict

In [63]:
def get_address_deposits_(deposit_transactions_df, tornado_addresses, pool_to_index_map):
    
    addresses_and_deposits_counts = pd.DataFrame(deposit_transactions_df[["from_address", "tornado_cash_address"]].value_counts()).rename(columns={0:"count"})
    
    addresses_and_deposits_dict = {}
    
    for row in addresses_and_deposits_counts.itertuples():
        if row.Index[0] in addresses_and_deposits_dict.keys():
            addresses_and_deposits_dict[row.Index[0]][tornado_addresses[row.Index[1]]] = row.count
        else:
            addresses_and_deposits_dict[row.Index[0]] = {tornado_addresses[row.Index[1]]: row.count}
    
    deposits_to_addresses_dict = {}
    
    # for transactions_dict in set(list(addresses_and_deposits_dict.values())):
    for transactions_dict in list(map(dict, set(tuple(sorted(sub.items())) for sub in addresses_and_deposits_dict.values()))):
        addresses = list(dict(filter(lambda elem: elem[1] == transactions_dict, addresses_and_deposits_dict.items())).keys())
        encoded_transaction = encode_transactions(transactions_dict, pool_to_index_map)
        deposits_to_addresses_dict[encoded_transaction] = addresses
            
    return deposits_to_addresses_dict

In [21]:
d = {'10 ETH': 10}
dict(filter(lambda elem: elem[1] == d, depo.items()))

def encode_transactions(transactions_dict, pool_to_index_map):
    encoded_transaction = np.zeros(len(pool_to_index_map.items()), dtype=int)
    for pool in transactions_dict.keys():
        encoded_transaction[pool_to_index_map[pool]] = transactions_dict[pool]
    return tuple(encoded_transaction)

In [64]:
get_address_deposits_(deposit_transactions_df, tornado_addresses, pool_to_index_map)

{(1,
  2,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0): ['0x64abc5b10c77d93be5038e7b4965d2f0d8279127', '0xba8ff620c4d53acfe79effa01d0fea13fc2ea7fd', '0x24dfbb120042a183a1878348d16b6c972cd193ae'],
 (3,
  7,
  6,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0): ['0x141bca806ab792d83e8d0886a78586a59a9bcfa1'],
 (1,
  0,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0): ['0xdd3a65c541aa16edf05b1a0e1f15179e1f1dc958',
  '0xa674029a4a58aaf028a7c9e498db2a21c8dbc8fb',
  '0x7066aa69b5e02b22d45dedd2ed3e5453289f3b9f',
  '0xdd6b514d6e19ed57f4dd457922a64192023e9b18',
  '0xf1ad946ae30a285d96ea396acd3fd22c5aa75ef8',
  '0x124c374ca7972e6f32caf4e14256ae13dc9413ee',
  '0x2452aeae3b64ab9415eb81da67fcbc7afaf91a01',
  '0x4a0bc7cb4435fc49e631f3d5a59ae6e39d7360a4',
  '0xd4e6fd78c21085d01dbd596d287360d746acf4d8',
  '0x945813970115558e0fba48330f7adb768ccfd556',
  '0xd8bebe411f0d19c8fea3363ddc9164d0050cf090

In [24]:
encode_transactions(d, pool_to_index_map)

(0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [48]:
d = {"10 ETH": 10, "1 ETH": 10}
set(d.values())

{10}

In [32]:
dict(filter(lambda elem: elem[1] == d, depo.items()))

{'0x682c1c8d0ff660ea8d9ecc16c7980fc1238a8777': {'10 ETH': 10},
 '0xbecac3ebed383b96df60cd2fb494930b74f548f0': {'10 ETH': 10},
 '0x7a5b1cbd4d1c9452f312568bba8d368a254b3f6b': {'10 ETH': 10},
 '0x823b92d6a4b2aed4b15675c7917c9f922ea8adad': {'10 ETH': 10},
 '0x4e2084adfd5f3cccb788189fc49988ab7a36e466': {'10 ETH': 10},
 '0x5a0d72127cf1aaf406a495b0ace882f399a47dad': {'10 ETH': 10},
 '0x5464eac9b4ed3b2084fbf3ac99ce1d9fdc3b48bc': {'10 ETH': 10},
 '0xba2f80f9be7c025c9fb025b88795859dcd0d290d': {'10 ETH': 10},
 '0xb0fad6fd44f48dce4a86caafd889c22bf16cc8b1': {'10 ETH': 10},
 '0x85476b1fa7b3b3c8db9462a2c6ec70c9b962fcf5': {'10 ETH': 10},
 '0xa0da83fcb4d921e966c67e747cdd66c4d60bb074': {'10 ETH': 10},
 '0x3e11d657331c286624826ac797a974777be0e47f': {'10 ETH': 10},
 '0x2fa2091f825625595149475a9df09cbc8d582457': {'10 ETH': 10},
 '0x3a3fd2ad9249bcf337c81eeeb6d950406ff8722c': {'10 ETH': 10},
 '0x05e7f8275cee815855ec30e8b1b056797d9c2cd1': {'10 ETH': 10},
 '0x2c980ff52df51f42823e3297cb3b7d40ad8f245f': {'10 ETH

In [25]:
depo = get_address_deposits(deposit_transactions_df, tornado_addresses)


In [6]:
pd.DataFrame(deposit_transactions_df[["from_address", "tornado_cash_address"]].value_counts()).rename(columns={0:"count"})

,,count
from_address,tornado_cash_address,
0x16e54b35d789832440ab47ae765e6a8098280676,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1068
0x35dd029618f4e1835008da21fd98850c776453f0,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,628
0xe906442c11b85acbc58eccb253b9a55a20b80a56,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,562
0xaf301de836c81deb8dff9dc22745e23c476155b2,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,503
0xdc4dcbc3f199a8b6ba51bdfd9db46051ab88a2f7,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,395
...,...,...
0x6b2dce929f92566ef5e8735d673937bd66911235,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1
0x6b29132ea388a308578c1d3be068d0e4fc9915a2,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1
0x6b224509ccad772634b2a2cecdcd7dcc770ec973,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1


In [25]:
address_deposits = get_address_deposits(deposit_transactions_df, tornado_addresses)
address_deposits

{'0x16e54b35d789832440ab47ae765e6a8098280676': {'0.1 ETH': 1068,
  '100 USDT': 1},
 '0x35dd029618f4e1835008da21fd98850c776453f0': {'0.1 ETH': 628},
 '0xe906442c11b85acbc58eccb253b9a55a20b80a56': {'0.1 ETH': 562},
 '0xaf301de836c81deb8dff9dc22745e23c476155b2': {'1 ETH': 503,
  '0.1 ETH': 96,
  '10 ETH': 8},
 '0xdc4dcbc3f199a8b6ba51bdfd9db46051ab88a2f7': {'1 ETH': 395},
 '0xb9d8ee59a0e5f908d3a87b8b272593761b40f298': {'1 ETH': 296},
 '0xd88189f7dee6e5dbd6cbc6f06fd357f4bf7f330b': {'100 ETH': 282, '10 WBTC': 80},
 '0x30030383d959675ec884e7ec88f05ee0f186cc06': {'100 ETH': 282},
 '0xe0e484dfa7f3aa36733a915d6f07eb5a57a74a11': {'100 ETH': 264,
  '100000 DAI': 50},
 '0x592c67dc41f6e5a7a39a836b031beb555cbef2f5': {'1 ETH': 251},
 '0x5386bb30475c4f6aac993faac62c3093ca0ac26d': {'1 ETH': 251},
 '0x6a1cf24c645db2e37141fa12e70cb67e56b336f3': {'0.1 ETH': 204, '10 ETH': 1},
 '0x0ad942b82c46cd22f4936ca7011e3a3c57755707': {'1 ETH': 202},
 '0xda8774d99fba48cfa13344285259401d93304a60': {'1 WBTC': 200},
 '0xa

In [9]:
d = {'1 ETH': 93,
  '100 ETH': 4,
  '1 WBTC': 3,
  '0.1 ETH': 1,
  '0.1 WBTC': 1}

In [8]:
def get_same_number_of_deposits(n_withdraws, deposit_transactions_df, tornado_addresses):

    address_deposits = get_address_deposits(deposit_transactions_df, tornado_addresses)
    
    return list(dict(filter(lambda elem: compare_transactions(n_withdraws, elem[1]), address_deposits.items())).keys())

In [8]:
def get_same_or_more_number_of_deposits(n_withdraws, address_deposits, tornado_addresses):
    
    return list(dict(filter(lambda elem: compare_transactions(n_withdraws, elem[1]), address_deposits.items())).keys())

In [26]:
get_same_or_more_number_of_deposits(d, address_deposits, tornado_addresses)

['0x6e8df51931f8675cf7e12447e1fe25b26c8775c7']

In [31]:
def same_number_of_deposits_heuristic(withdraw_transaction, address_deposits, withdraw_transactions_df, tornado_addresses):
    n_withdraws = get_number_of_withdraws(withdraw_transaction, withdraw_transactions_df, tornado_addresses)

    C = get_same_or_more_number_of_deposits(n_withdraws, address_deposits, tornado_addresses)
    
    try:
        return 1 - 1/len(C)
    except:
        return 1

In [27]:
withdraw_transaction = withdraw_transactions_df.iloc[21]

In [23]:
get_number_of_withdraws(withdraw_transaction, withdraw_transactions_df, tornado_addresses)

{'10 ETH': 2, '1 ETH': 2, '100 DAI': 1}

In [28]:
same_number_of_deposits_heuristic(withdraw_transaction, address_deposits, withdraw_transactions_df, tornado_addresses)

{'0.1 ETH': 1}


0.9997776295307983

In [29]:
def apply_same_number_of_deposits_heuristic(deposit_transactions_df, withdraw_transactions_df, tornado_addresses):
    
    address_deposits = get_address_deposits(deposit_transactions_df, tornado_addresses)
    
    tx_hash_and_anonimity_score = {}
    
    for withdraw_row in tqdm(withdraw_transactions_df.itertuples(), total = withdraw_transactions_df.shape[0], mininterval=5):
        anonimity_score = same_number_of_deposits_heuristic(withdraw_row, address_deposits, withdraw_transactions_df, tornado_addresses)
        tx_hash_and_anonimity_score[withdraw_row.hash] = anonimity_score

    return tx_hash_and_anonimity_score

In [43]:
scores = apply_same_number_of_deposits_heuristic(deposit_transactions_df, repeated_withdraw_transactions_by_address, tornado_addresses)

  1%|▋                                                   | 1025/82947 [03:52<5:09:23,  4.41it/s]


KeyboardInterrupt: 